# Basic Data Architecture

<img src='job_architecture.png'>

# Data Engineering - ETL

## Librairies

In [2]:
import pandas as pd
import numpy as np
import sqlite3

## Extract

In [3]:
url = 'http://app-sellaci.eu-central-1.elasticbeanstalk.com/hospital_data?collection={collection}'

df_physicians_extracted = pd.read_json(url.format(collection='physicians'))
df_consultations_extracted = pd.read_json(url.format(collection='consultations'))
df_patients_extracted = pd.read_json(url.format(collection='patients'))

print('physicians')
display(df_physicians_extracted.tail(1))
print('consultations')
display(df_consultations_extracted.tail(1))
print('patients')
display(df_patients_extracted.tail(1))

physicians


,birth_date,graduated_date,id,name,renowned_coefficient,retired_date,speciality
499,NaN,2020-03-10,129,Austin Knight,16,2020-09-16,"[digestive-and-visceral-surgeon, 1, 0]"


consultations


,device_os,duration,id,is_free_act,patient_id,physician_id,price,start_date,status
5047,___ ip 7WebApp 283,10,3459,0,73146,303,25.0,2020-07-06 12:10:00,closed


patients


,birth_date,created_date,gender,id,last_vist_date
93124,1970-05-01,2020-10-31 17:58:24.158000,F,40573,2020-10-31 19:24:43.944000


## Transform

In [4]:
df_physicians_extracted[['speciality_cleaned', 'is_doctor', 'is_mental']] = (
    pd.DataFrame(df_physicians_extracted['speciality'].tolist(), index=df_physicians_extracted.index)
)
df_physicians_extracted.tail()

,birth_date,graduated_date,id,name,renowned_coefficient,retired_date,speciality,speciality_cleaned,is_doctor,is_mental
495,NaN,2020-01-10,24,John Johnson,23,None,"[occupational-therapist, 0, 1]",occupational-therapist,0,1
496,NaN,2019-12-18,53,Steven Petersen,26,2020-12-02,"[orthodontist, 1, 0]",orthodontist,1,0
497,NaN,2020-03-27,240,Sharon Frazier,14,2020-12-02,"[orthodontist, 1, 0]",orthodontist,1,0
498,NaN,2020-03-20,265,Gary Flowers,5,2020-12-02,"[orthodontist, 1, 0]",orthodontist,1,0
499,NaN,2020-03-10,129,Austin Knight,16,2020-09-16,"[digestive-and-visceral-surgeon, 1, 0]",digestive-and-visceral-surgeon,1,0


In [5]:
df_specialities_cleaned  = (
    df_physicians_extracted[['speciality_cleaned', 'is_doctor', 'is_mental']]\
        .drop_duplicates()\
        .reset_index(drop=True)\
        .rename(columns={'speciality_cleaned': "label"})
)
df_specialities_cleaned.tail()

,label,is_doctor,is_mental
40,osteopath,0,0
41,psychotherapist,0,1
42,occupational-therapist,0,1
43,orthodontist,1,0
44,digestive-and-visceral-surgeon,1,0


In [6]:
# remove birthdate because column is empty
df_physicians_cleaned = df_physicians_extracted[[
    'id',
    'name',
    'graduated_date',
    'renowned_coefficient',
    'retired_date',
    'speciality_cleaned',
]].rename(columns={'speciality_cleaned': "speciality"})

df_physicians_cleaned.tail()

,id,name,graduated_date,renowned_coefficient,retired_date,speciality
495,24,John Johnson,2020-01-10,23,None,occupational-therapist
496,53,Steven Petersen,2019-12-18,26,2020-12-02,orthodontist
497,240,Sharon Frazier,2020-03-27,14,2020-12-02,orthodontist
498,265,Gary Flowers,2020-03-20,5,2020-12-02,orthodontist
499,129,Austin Knight,2020-03-10,16,2020-09-16,digestive-and-visceral-surgeon


In [7]:
df_consultations_extracted['device_os_cleaned'] = df_consultations_extracted['device_os'].apply(
    lambda x: x.split(' ')[2].replace('7', '') if type(x) == str else np.nan
)
df_consultations_extracted

,device_os,duration,id,is_free_act,patient_id,physician_id,price,start_date,status,device_os_cleaned
0,___ ip 7WebApp 269,15,2350,0,46026,289,25.0,2020-03-27 09:00:00,closed,WebApp
1,___ ip 7Android 406,20,2300,0,50611,426,25.0,2020-04-15 15:00:00,closed,Android
2,___ ip 7iOS 65,15,1277,0,44851,85,0.0,2020-02-10 09:45:00,closed,iOS
3,___ ip 7WebApp 389,60,3334,0,43742,409,30.0,2020-02-10 19:00:00,closed,WebApp
4,None,15,1387,0,30469,190,25.0,2020-06-26 19:00:00,closed,NaN
...,...,...,...,...,...,...,...,...,...,...
5043,___ ip 7Android 265,20,782,0,72508,285,30.0,2020-05-01 12:00:00,closed,Android
5044,___ ip 7Android 189,15,1309,0,80446,209,25.0,2020-03-11 12:00:00,closed,Android
5045,___ ip 7WebApp 281,15,774,0,29362,301,0.0,2020-03-26 11:00:00,closed,WebApp
5046,___ ip 7iOS 329,15,1519,0,30034,349,25.0,2020-07-17 09:30:00,closed,iOS


In [8]:
df_consultations_cleaned = df_consultations_extracted[[
    'id',
    'patient_id',
    'physician_id',
    'start_date',
    'status',
    'duration',
    'price',
    'is_free_act',
    'device_os_cleaned',
]].rename(
    columns={'device_os_cleaned': 'device_os'}
)

df_consultations_cleaned.tail()

,id,patient_id,physician_id,start_date,status,duration,price,is_free_act,device_os
5043,782,72508,285,2020-05-01 12:00:00,closed,20,30.0,0,Android
5044,1309,80446,209,2020-03-11 12:00:00,closed,15,25.0,0,Android
5045,774,29362,301,2020-03-26 11:00:00,closed,15,0.0,0,WebApp
5046,1519,30034,349,2020-07-17 09:30:00,closed,15,25.0,0,iOS
5047,3459,73146,303,2020-07-06 12:10:00,closed,10,25.0,0,WebApp


In [9]:
df_patients_cleaned = df_patients_extracted
df_patients_cleaned.tail()

,birth_date,created_date,gender,id,last_vist_date
93120,1976-06-01,2020-10-31 12:42:57.271000,F,69202,2020-10-31 14:48:39.502000
93121,1987-06-01,2020-10-31 14:44:51.580000,M,86021,2020-10-31 16:05:12.287000
93122,1983-09-01,2020-10-31 15:05:46.311000,F,88640,2020-10-31 16:12:52.548000
93123,2015-03-01,2020-10-31 15:07:07.453000,F,49466,2020-10-31 21:53:04.072000
93124,1970-05-01,2020-10-31 17:58:24.158000,F,40573,2020-10-31 19:24:43.944000


## Load

In [10]:
conn = sqlite3.connect('hospitals_database.db')
c = conn.cursor()

c.execute(f"CREATE TABLE IF NOT EXISTS specialities ({', '.join(df_specialities_cleaned.columns)})")
conn.commit()
df_specialities_cleaned.to_sql('specialities', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS physicians ({', '.join(df_physicians_cleaned.columns)})")
conn.commit()
df_physicians_cleaned.to_sql('physicians', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS consultations ({', '.join(df_consultations_cleaned.columns)})")
conn.commit()
df_consultations_cleaned.to_sql('consultations', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS patients ({', '.join(df_patients_cleaned.columns)})")
conn.commit()
df_patients_cleaned.to_sql('patients', conn, if_exists='replace', index = False)

93125

# The Database Schema

<img src='schema.png'>

Read 1 to N.  
Ex: <i>"1 physician can do N consultations"</i>

# Analysis

In [11]:
con = sqlite3.connect('hospitals_database.db')

# SELECT
<b>Display Physicians overview</b>

In [12]:
pd.read_sql(
    '''
        SELECT 
            *
        FROM 
            physicians
    ''', 
    con
)

,id,name,graduated_date,renowned_coefficient,retired_date,speciality
0,88,Claudia Washington,2017-01-03,422,None,dentist
1,78,Angela Francis,2019-01-26,245,None,dentist
2,238,Nicole Horne,2019-07-19,1261,None,dentist
3,283,Nancy Valencia,2020-01-06,31,None,dentist
4,164,Mason Campbell,2019-11-05,3245,None,dentist
...,...,...,...,...,...,...
495,24,John Johnson,2020-01-10,23,None,occupational-therapist
496,53,Steven Petersen,2019-12-18,26,2020-12-02,orthodontist
497,240,Sharon Frazier,2020-03-27,14,2020-12-02,orthodontist
498,265,Gary Flowers,2020-03-20,5,2020-12-02,orthodontist


<b>Display Physicians Names and Specialities</b>

In [13]:
pd.read_sql(
    '''
        SELECT 
            name, 
            speciality
        FROM 
            physicians
        LIMIT 5
    ''', 
    con
)

,name,speciality
0,Claudia Washington,dentist
1,Angela Francis,dentist
2,Nicole Horne,dentist
3,Nancy Valencia,dentist
4,Mason Campbell,dentist


# WHERE
## Simple
<b>Display General Practician names and speciality</b>

In [14]:
pd.read_sql(
    '''
        SELECT 
            name, 
            speciality
        FROM 
            physicians
        WHERE
            speciality='general-practician'
        LIMIT 5
    ''', 
    con
)

,name,speciality
0,Joanne Wright,general-practician
1,Kevin Cain,general-practician
2,Kimberly Stuart DDS,general-practician
3,Kelly Smith,general-practician
4,Elizabeth Hill,general-practician


## AND
<b>Display General Practician with Renowned Coefficient above 1k names and speciality</b>

In [15]:
pd.read_sql(
    '''
        SELECT 
            name, 
            speciality,
            renowned_coefficient
        FROM 
            physicians
        WHERE
            speciality='general-practician'
            AND renowned_coefficient > 1000
        LIMIT 10
    ''', 
    con
)

,name,speciality,renowned_coefficient
0,Joanne Wright,general-practician,4392
1,Kelly Smith,general-practician,10165
2,Shane Ward,general-practician,1487
3,Sarah Klein,general-practician,3983
4,Ashley Guzman,general-practician,3521
5,William Wilson,general-practician,2440
6,Mary Morris,general-practician,2809
7,James Foster,general-practician,4391
8,Ann Murphy,general-practician,1257
9,Tiffany Jones,general-practician,1925


## OR
<b>Display General Practician names and speciality with either Renowned Coefficient above 1k OR gradute date before january 2018 </b>

In [16]:
pd.read_sql(
    '''
        SELECT 
            name, 
            speciality,
            renowned_coefficient
        FROM 
            physicians
        WHERE
            speciality='general-practician'
            AND (renowned_coefficient > 1000 OR graduated_date < 2019-01-01)
        LIMIT 10
    ''', 
    con
)

,name,speciality,renowned_coefficient
0,Joanne Wright,general-practician,4392
1,Kelly Smith,general-practician,10165
2,Shane Ward,general-practician,1487
3,Sarah Klein,general-practician,3983
4,Ashley Guzman,general-practician,3521
5,William Wilson,general-practician,2440
6,Mary Morris,general-practician,2809
7,James Foster,general-practician,4391
8,Ann Murphy,general-practician,1257
9,Tiffany Jones,general-practician,1925


## LIKE
<b>Display General Practician with name starting by "Cha"</b>

In [17]:
pd.read_sql(
    '''
        SELECT 
            name, 
            speciality,
            renowned_coefficient
        FROM 
            physicians
        WHERE
            speciality='general-practician'
            AND name LIKE 'Cha%'
        LIMIT 5
    ''', 
    con
)

,name,speciality,renowned_coefficient
0,Charles Brown,general-practician,343


## BETWEEN
<b>Display Physicians graduated in March 2019</b>

In [18]:
pd.read_sql(
    '''
        SELECT 
            name, 
            graduated_date
        FROM 
            physicians
        WHERE
            graduated_date BETWEEN '2020-03-01' AND '2020-04-01'
        LIMIT 5
    ''', 
    con
)

,name,graduated_date
0,Devin Bautista,2020-03-27
1,Richard Barker,2020-03-24
2,Frank Cox,2020-03-18
3,Russell Welch,2020-03-24
4,Dale Gonzalez,2020-03-13


## NULL
<b>Display Physicians still active</b>

In [19]:
pd.read_sql(
    '''
        SELECT 
            name, 
            graduated_date
        FROM 
            physicians
        WHERE
            retired_date IS NULL
        LIMIT 5
    ''', 
    con
)

,name,graduated_date
0,Claudia Washington,2017-01-03
1,Angela Francis,2019-01-26
2,Nicole Horne,2019-07-19
3,Nancy Valencia,2020-01-06
4,Mason Campbell,2019-11-05


# ORDER
<b>Display 2 highest Renowned coefficient GP (general practician)</b>

In [20]:
pd.read_sql(
    '''
        SELECT 
            name, 
            speciality,
            renowned_coefficient
        FROM 
            physicians
        WHERE
            speciality='general-practician'
            AND renowned_coefficient > 1000
        ORDER BY renowned_coefficient DESC
        LIMIT 2
    ''', 
    con
)

,name,speciality,renowned_coefficient
0,Melissa Baldwin,general-practician,23627
1,Geoffrey Phillips,general-practician,16543


# AGGREGATING FUNCTIONS
## COUNT / MIN / MAX / AVG
<b>Get number of physicians in physician table</b>  
<b>Get older physician, last retired and average renowned in physician table</b>

In [21]:
pd.read_sql(
    '''
        SELECT 
            COUNT(*) AS nb_rows_physicians,
            MAX(retired_date) AS last_retired_date,
            AVG(renowned_coefficient) AS avg_renowned
        FROM 
            physicians
    ''', 
    con
)

,nb_rows_physicians,last_retired_date,avg_renowned
0,500,2021-08-11,1172.698


And so many other aggregation functions !

# GROUP BY 
## COUNT
<b>Get number of physicians not retired by speciality</b>

In [22]:
pd.read_sql(
    '''
        SELECT 
            speciality,
            count(*) AS nb_physicians
        FROM 
            physicians
        WHERE
            retired_date IS NULL
        GROUP BY
            speciality
        ORDER BY 
            nb_physicians DESC
        LIMIT 5
    ''', 
    con
)

,speciality,nb_physicians
0,general-practician,156
1,psychologist,31
2,dietitian,24
3,physical-therapist,22
4,pediatrician,11


# HAVING
<b>Then where after Group By !</b>
<b>Get number of physicians not retired by speciality with more than 25 physicians</b>

In [23]:
pd.read_sql(
    '''
        SELECT 
            speciality,
            count(*) AS nb_physicians
        FROM 
            physicians
        WHERE
            retired_date IS NULL
        GROUP BY
            speciality
        HAVING 
            nb_physicians > 25
        ORDER BY 
            nb_physicians DESC
        LIMIT 5
    ''', 
    con
)

,speciality,nb_physicians
0,general-practician,156
1,psychologist,31


# JOIN 

<img src='joins.png'>

## INNER JOIN
<b>Get number of consultations made by physicians who consulted</b>

In [24]:
pd.read_sql(
    '''
        SELECT 
            p.name,
            count(*) AS nb_consultations
        FROM 
            physicians p
        INNER JOIN 
            consultations c ON p.id = c.physician_id
        GROUP BY
            p.id, p.name
        ORDER BY 
            nb_consultations DESC 
        LIMIT 5
    ''', 
    con
)

,name,nb_consultations
0,Melissa Baldwin,170
1,Kayla Stone,150
2,Donna Walker,135
3,William Salas,133
4,Karen Stone,133


## LEFT JOIN
<b>Get number of consultations made by physicians</b>

In [25]:
pd.read_sql(
    '''
        SELECT 
            p.id, 
            COUNT(c.id) as nb_consultations
        FROM 
            physicians p
        LEFT JOIN 
            consultations c ON c.physician_id = p.id
        GROUP BY 
            p.id
        ORDER BY 
            nb_consultations DESC
        LIMIT 5
    ''', 
    con
)

,id,nb_consultations
0,31,170
1,84,150
2,349,135
3,303,133
4,86,133


## UNION

In [26]:
pd.read_sql(
    '''
        SELECT 
            'mental' AS speciality,
            COUNT(*) AS nb_consultations
        FROM 
            consultations c 
            INNER JOIN physicians p ON c.physician_id = p.id
            INNER JOIN specialities s ON s.label = p.speciality
            WHERE s.is_mental = True
            
        UNION
        
        SELECT 
            'others' AS speciality,
            COUNT(*) AS nb_consultations
        FROM 
            consultations c 
            INNER JOIN physicians p ON c.physician_id = p.id
            INNER JOIN specialities s ON s.label = p.speciality
            WHERE s.is_mental = False    
    ''', 
    con
)

,speciality,nb_consultations
0,mental,157
1,others,4891


Check CROSS JOIN & OUTER JOIN on your side

# CTE - Common Table Expression

In [27]:
pd.read_sql(
    '''
        WITH mental_consultations AS (
            SELECT 
                'mental' AS speciality,
                COUNT(*) AS nb_consultations
            FROM 
                consultations c 
                INNER JOIN physicians p ON c.physician_id = p.id
                INNER JOIN specialities s ON s.label = p.speciality
                WHERE s.is_mental = True
        ),
                    
        physical_consultations AS (
            SELECT 
                'others' AS speciality,
                COUNT(*) AS nb_consultations
            FROM 
                consultations c 
                INNER JOIN physicians p ON c.physician_id = p.id
                INNER JOIN specialities s ON s.label = p.speciality
                WHERE s.is_mental = False    
        )
        
        SELECT 
            *
        FROM
            mental_consultations 
        
        UNION
        
        SELECT 
            *
        FROM 
            physical_consultations
    ''', 
    con
)

,speciality,nb_consultations
0,mental,157
1,others,4891


# Business Analyst Question

<b>I'm the hospital Director. Give me 10 KPIs I should follow every week.</b>  
 1.- Number of pacients.\
 2.- Number of consultations.\
 3.- Number of physicians.\
 4.- Time spent at the hospital an average.\
 5.- Occupation rate at the hospital.\
 6.- Busiest days of the week.\
 7.- Number of specialities.\
 8.- Consultations per speciality.\
 9.- Availability vs demand.\
 10.-Revenue per day\
 \
<b>I'm the hospital Data Engineer. Give me 5 KPIs I should follow to check my data veracity.</b>  
 1.- Mistakes by entries of workers at the hospital\
 2.- Data decay\
 3.- Ratio of errors\
 4.- Data transformation errors\
 5.- Number of duplicate cases\

# SQL Queries

## Consultations
<b>Get number of consultations by week</b>    
<b>Get number of consultations by month for GP (general practicians)</b> 

## Physicians 
<b>Number of new physicians by week</b> 
<b>Number of retired physicians by week</b> 

## Patients
<b>Number of new patients who are in the hospital database by week</b>  
<b>Number of patients who consulted by week</b>  
<b>Number of new patients who consulted by week</b>  
<b>Number of patients who came back for a consultation by week</b>  

## Others
<b>Get price earn by the hospital by week</b>   
<b>Get raito #price earned / #consultations made</b>  
<b>Get most paid physician by month</b>  
<b>Get physician having TC the most by week</b>  
<b>Get most prorfitable speciality by month</b>   
<b>Get top 10 medication ever</b>   
<b>Get physician who have the highet prescription/consultation ratio</b>   
<b>How many consultation did physicians who retired ?</b>   
<b>Should we call back a physican ? If yes, who ?</b>   
<b>How long a physian stay in the hospital in average ?</b>   

## Usage
<b>Which device is the most used for booking consultations ?</b>  
<b>Do we have physicians who never consulted ?</b>  
<b>Physician doing too many free act ?</b>   
<b>How many times the best physican/patient couple consulted togeter ?</b>   
<b>What is the median age difference between a physician and its patient ?</b>   

## Consultations

In [28]:
# Get number of consultations by week
pd.read_sql(
    '''
        SELECT 
            strftime('%W',start_date) AS weeks,
            COUNT(*) AS consultations_done
        FROM 
            consultations
        GROUP BY strftime('%W',start_date)
    ''', 
    con
)

,weeks,consultations_done
0,00,51
1,01,80
2,02,80
3,03,103
4,04,102
5,05,111
6,06,107
7,07,97
8,08,93
9,09,150


In [29]:
# Get number of consultations by month for GP (general practicians)
pd.read_sql(
    '''
        SELECT 
            strftime('%Y %m',start_date) AS months,
            COUNT(*) AS consultations_done
        FROM 
            consultations c 
            LEFT JOIN physicians p ON c.physician_id = p.id
        WHERE 
            p.speciality like '%general%'
        GROUP BY
            strftime('%Y %m',start_date)
        
    ''',
    con
)

,months,consultations_done
0,2020 01,335
1,2020 02,364
2,2020 03,1117
3,2020 04,518
4,2020 05,314
5,2020 06,197
6,2020 07,262
7,2020 08,258
8,2020 09,294
9,2020 10,328


## Physicians

In [30]:
# Number of new physicians by week
pd.read_sql(
    '''
        SELECT 
            strftime('%Y %W',graduated_date) AS date,
            COUNT(*) AS new_physicians
        FROM 
            physicians
        WHERE
            retired_date IS NULL
        GROUP BY
            strftime('%Y %W',graduated_date)
    ''',
    con
)

,date,new_physicians
0,None,4
1,2017 01,1
2,2017 09,4
3,2017 11,2
4,2017 12,1
...,...,...
89,2020 49,1
90,2021 10,1
91,2021 18,1
92,2021 20,1


In [31]:
# Number of retired physicians by week
pd.read_sql(
    '''
        SELECT 
            strftime('%Y %W',graduated_date) AS date,
            COUNT(*) AS retired
        FROM 
            physicians
        WHERE
            retired_date IS NOT NULL
        GROUP BY
            strftime('%Y %W',graduated_date)
    ''',
    con
)

,date,retired
0,2017 01,1
1,2017 09,1
2,2017 12,3
3,2017 13,3
4,2017 15,2
...,...,...
57,2020 11,14
58,2020 12,8
59,2020 13,1
60,2020 14,1


## Patients

In [32]:
# Number of new patients who are in the hospital database by week
pd.read_sql(
    '''
        SELECT 
            strftime('%Y %W',created_date) AS date,
            COUNT(DISTINCT(id)) AS new_patient
        FROM 
            patients
        GROUP BY
            strftime('%Y %W',created_date)
    ''',
    con
)

,date,new_patient
0,2017 11,1
1,2017 12,2
2,2017 13,1
3,2017 14,1
4,2017 16,3
...,...,...
181,2020 39,1157
182,2020 40,1250
183,2020 41,1249
184,2020 42,1304


In [33]:
# Number of patients who consulted by week
pd.read_sql(
    '''
        SELECT 
            strftime('%Y %W',start_date) AS date,
            COUNT(DISTINCT(patient_id)) AS patients_consulted
        FROM 
            consultations
        GROUP BY
            strftime('%Y %W',start_date)
    ''',
    con
)

,date,patients_consulted
0,2020 00,51
1,2020 01,79
2,2020 02,80
3,2020 03,102
4,2020 04,101
5,2020 05,110
6,2020 06,106
7,2020 07,95
8,2020 08,90
9,2020 09,145


In [34]:
# Number of new patients who consulted by week
pd.read_sql(
    '''
        SELECT 
            strftime('%Y %W',created_date) AS date,
            COUNT(DISTINCT(p.id)) AS new_patient
        FROM 
            patients p
            INNER JOIN consultations c ON c.patient_id = p.id
        WHERE
            DATE(c.start_date) == DATE(p.created_date)
        GROUP BY
            strftime('%Y %W',created_date)

    ''',
    con
)

,date,new_patient
0,2020 00,13
1,2020 01,39
2,2020 02,26
3,2020 03,43
4,2020 04,36
5,2020 05,40
6,2020 06,33
7,2020 07,39
8,2020 08,33
9,2020 09,56


In [35]:
# Number of patients who came back for a consultation by week
pd.read_sql(
    '''
        SELECT 
            strftime('%Y %W',start_date) AS date,
            COUNT(DISTINCT(p.id)) AS new_patient
        FROM 
            patients p
            INNER JOIN consultations c ON c.patient_id = p.id
        WHERE
            DATE(c.start_date) <> DATE(p.created_date)
        GROUP BY
            strftime('%Y %W',created_date)

    ''',
    con
)

,date,new_patient
0,2020 09,1
1,2020 04,1
2,2020 43,1
3,2020 00,1
4,2020 18,1
...,...,...
128,2020 39,13
129,2020 43,25
130,2020 41,25
131,2020 43,23


## Others

In [36]:
#Get price earn by the hospital by week
pd.read_sql(
    '''
        SELECT 
            strftime('%Y %W',start_date) AS date,
            SUM(price) AS price_per_week
        FROM 
            consultations
        GROUP BY
            strftime('%Y %W',start_date)
    ''',
    con
)

,date,price_per_week
0,2020 00,100.00
1,2020 01,170.00
2,2020 02,105.00
3,2020 03,290.00
4,2020 04,780.00
5,2020 05,185.00
6,2020 06,160.00
7,2020 07,170.00
8,2020 08,215.00
9,2020 09,1986.00


In [48]:
#Get ratio (price earned / #consultations made
pd.read_sql(
    '''
        SELECT 
            SUM(price) AS total_price,
            SUM(id) AS total_consultations,
            (SUM(price) / SUM(id))*100 AS ratio
        FROM 
            consultations
    ''',
    con
)


,total_price,total_consultations,ratio
0,82957.91,12738628,0.651231


In [81]:
#Get most paid physician by month
pd.read_sql(
    '''
        SELECT 
            date, physician_id, max(most_paid)
        FROM (
            SELECT 
                strftime('%Y %m',start_date) AS date,
                physician_id,
                sum(price) AS most_paid
            FROM 
                consultations
            GROUP BY
                strftime('%Y %m',start_date),
                physician_id
                )
        GROUP BY
                date
    ''',
    con
)

,date,physician_id,max(most_paid)
0,2020 01,492,200.0
1,2020 02,492,120.0
2,2020 03,86,1035.0
3,2020 04,31,575.0
4,2020 05,84,600.0
5,2020 06,227,448.0
6,2020 07,227,544.0
7,2020 08,306,650.0
8,2020 09,349,525.0
9,2020 10,232,475.0


In [83]:
#Get physician having TC the most by week ?????
pd.read_sql(
    '''
        SELECT 
            date, physician_id, max(most_consultations)
        FROM (
            SELECT 
                strftime('%Y %W',start_date) AS date,
                physician_id,
                COUNT(ID) AS most_consultations
            FROM 
                consultations
            GROUP BY
                strftime('%Y %W',start_date),
                physician_id
                )
        GROUP BY
                date
    ''',
    con
)

,date,physician_id,max(most_consultations)
0,2020 00,31,5
1,2020 01,86,7
2,2020 02,289,7
3,2020 03,32,6
4,2020 04,158,7
5,2020 05,86,10
6,2020 06,289,8
7,2020 07,31,5
8,2020 08,31,5
9,2020 09,86,13


In [91]:
# Get most profitable speciality by month
pd.read_sql(
    '''
        SELECT 
            date, speciality, max(most_paid)
        FROM (
            SELECT 
                strftime('%Y %m',c.start_date) AS date,
                p.speciality,
                sum(c.price) AS most_paid
            FROM 
                consultations AS c
                INNER JOIN physicians AS P ON p.id = c.physician_id
            GROUP BY
                strftime('%Y %m',c.start_date),
                c.physician_id
                )
        GROUP BY
                date
    ''',
    con
)

,date,speciality,max(most_paid)
0,2020 01,orthophonist,200.0
1,2020 02,orthophonist,120.0
2,2020 03,general-practician,1035.0
3,2020 04,general-practician,575.0
4,2020 05,general-practician,600.0
5,2020 06,pediatrician,448.0
6,2020 07,pediatrician,544.0
7,2020 08,general-practician,650.0
8,2020 09,general-practician,525.0
9,2020 10,general-practician,475.0


In [303]:
# Get top 10 medication ever
#Get physician who have the highet prescription/consultation ratio
print('These two questions cannot be solved as we do not have enough data available. Medications and prescriptions are not provided')

These two questions cannot be solved as we do not have enough data available. Medications and prescriptions are not provided


In [103]:
# How many consultation did physicians who retired ?
pd.read_sql(
    '''
        SELECT 
            sum(retired_physicians) AS retired_doctors,
            sum(consultations_retired) AS Num_consultations_made
        FROM (SELECT 
                c.physician_id,
                COUNT(c.id) AS consultations_retired,
                COUNT(DISTINCT(p.id)) AS retired_physicians
            FROM 
                consultations c
                INNER JOIN physicians AS P ON p.id = c.physician_id
            WHERE
                retired_date IS NOT NULL
            GROUP BY
                c.physician_id
                )
    ''',
    con
)

,retired_doctors,Num_consultations_made
0,74,504


In [108]:
# Should we call back a physican ? If yes, who ?
pd.read_sql(
    '''
        SELECT 
            id,
            name
        FROM 
            physicians 
        WHERE
            retired_date IS NOT NULL
    ''',
    con
)

,id,name
0,391,Stacy Pearson
1,69,Kenneth Watkins
2,384,Amanda Fox
3,0,Dawn Rodriguez
4,287,Carlos Jackson
...,...,...
170,1,Kimberly Davis
171,53,Steven Petersen
172,240,Sharon Frazier
173,265,Gary Flowers


In [140]:
# How long a physian stay in the hospital in average ? Per day
pd.read_sql(
    '''
        SELECT 
            date,
            minutes_consultations/phys
        FROM (
            SELECT 
                strftime('%Y %m %d',c.start_date) AS date,
                SUM(duration) AS minutes_consultations,
                COUNT(physician_id) AS phys
            FROM 
                consultations AS c
                INNER JOIN physicians AS P ON p.id = c.physician_id
            GROUP BY
                strftime('%Y %m %d',c.start_date)
                )
        GROUP BY
                date
    ''',
    con
)

,date,minutes_consultations/phys
0,2020 01 01,15
1,2020 01 02,25
2,2020 01 03,17
3,2020 01 04,21
4,2020 01 05,20
...,...,...
300,2020 10 27,13
301,2020 10 28,14
302,2020 10 29,13
303,2020 10 30,18


In [151]:
# How long a physian stay in the hospital in average ? overall
pd.read_sql(
    '''
        SELECT 
            AVG(minutes_consultations/phys) AS overall_minutes_per_day
        FROM (
            SELECT 
                strftime('%Y %m %d',c.start_date) AS date,
                SUM(duration) AS minutes_consultations,
                COUNT(physician_id) AS phys
            FROM 
                consultations AS c
                INNER JOIN physicians AS P ON p.id = c.physician_id
            GROUP BY
                strftime('%Y %m %d',c.start_date)
                )
    ''',
    con
)

,overall_minutes_per_day
0,15.701639


## Usage

In [159]:
#Which device is the most used for booking consultations ?
pd.read_sql(
    '''
        SELECT 
            device_os,
            COUNT(device_os) AS operative
        FROM 
            consultations
        GROUP BY
            device_os
        ORDER BY operative DESC
    ''',
    con
)

,device_os,operative
0,Android,1732
1,iOS,1714
2,WebApp,970
3,None,0


In [166]:
# Do we have physicians who never consulted ?
pd.read_sql(
    '''
        SELECT 
             COUNT(DISTINCT(p.id)),
             COUNT(DISTINCT(physician_id))
        FROM 
             consultations AS c
             INNER JOIN physicians AS P ON p.id = c.physician_id
    ''',
    con
)

,COUNT(DISTINCT(p.id)),COUNT(DISTINCT(physician_id))
0,281,281


In [173]:
# Do we have physicians who never consulted ?
pd.read_sql(
    '''
        SELECT 
            physician_id
        FROM   
            consultations
        WHERE  NOT EXISTS (SELECT 
                                *
                           FROM   
                               physicians
                           WHERE  
                               consultations.physician_id = physicians.id)
    ''',
    con
)

,physician_id


In [177]:
# Physician doing too many free act ?
pd.read_sql(
    '''
        SELECT 
            physician_id,
            SUM(is_free_act) AS Free_act
        FROM 
            consultations
        GROUP BY
            physician_id
        ORDER BY Free_act DESC
        LIMIT 1
    ''',
    con
)

,physician_id,Free_act
0,31,20


In [273]:
# How many times the best physican/patient couple consulted togeter ? A
pd.read_sql(
    '''
        SELECT
            id,
            name,
            patient_id,
            MAX(conteo) AS times_visited,
            best_coefficient
        FROM (
        
            SELECT 
                P.id AS id,
                name,
                MAX(renowned_coefficient) AS best_coefficient,
                patient_id,
                COUNT(patient_id) AS conteo
            FROM 
                consultations c
                INNER JOIN physicians AS p ON p.id = c.physician_id
            GROUP BY patient_id
            HAVING renowned_coefficient > 32870
        )
        
    ''',
    con
)

,id,name,patient_id,times_visited,best_coefficient
0,227,Steven Walker,16349,2,32874


In [278]:
# How many times the best physican/patient couple consulted togeter ? B
pd.read_sql(
    '''
        SELECT
            id,
            name,
            patient_id,
            MAX(conteo) AS times_visited,
            best_coefficient
        FROM (
        
            SELECT 
                P.id AS id,
                name,
                MAX(renowned_coefficient) AS best_coefficient,
                patient_id,
                COUNT(patient_id) AS conteo
            FROM 
                consultations c
                INNER JOIN physicians AS p ON p.id = c.physician_id
            GROUP BY patient_id
        )
        
    ''',
    con
)

,id,name,patient_id,times_visited,best_coefficient
0,407,Donald Hopkins,66418,10,5114


In [302]:
# What is the median age difference between a physician and its patient ?
pd.read_sql(
    '''
        SELECT 
            DISTINCT(pa.id) AS patient_id,
            pa.birth_date AS patient_birthdate
        FROM 
            patients pa
            INNER JOIN consultations c ON pa.id = c.patient_id
            INNER JOIN physicians ph ON ph.id = c.physician_id
            
    ''',
    con
)

print('Median cannot be calculated as we do not have enough data. Birth date of physician is not provided.')

Median cannot be calculated as we do not have enough data. Birth date of physician is not provided.
